In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM


In [ ]:
!pip install transformers

model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

In [ ]:
pip install transformers peft datasets accelerate torch

In [ ]:
from datasets import load_dataset

dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

In [ ]:

def preprocess_function(examples):
    inputs = [f"{q} {a}" for q, a in zip(examples["input"], examples["output"])]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # Prepare labels as well (outputs)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=512, padding="max_length", truncation=True)

    # Add labels to the tokenized inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [6]:
from peft import get_peft_model, LoraConfig

config = LoraConfig(
    r=1,
    lora_alpha=32,
    lora_dropout=0.1,
    base_model_name_or_path="microsoft/Phi-3.5-mini-instruct",
    target_modules=[
        "self_attn.qkv_proj",        # Query, key, value projections in attention
        "self_attn.o_proj",          # Output projection in attention
        "mlp.gate_up_proj",          # MLP up-projection
        "mlp.down_proj"              # MLP down-projection
    ]
)
model = get_peft_model(model, config)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    remove_unused_columns=False,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to='none',
    logging_dir='./logs',
    save_total_limit=2
)

In [ ]:
tokenized_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)  # 10% for test
print(tokenized_dataset.keys())   9

# Proceed with Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

In [ ]:
trainer.train()